In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 

from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('core-nomad-381608-a81e95959692.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

#
date = 0 #当日は0、前日は1・・・
#date = 1 #当日は0、前日は1・・・
date2 = "08/13"
pmc = 27090010 # 店舗ID
SPREADSHEET_KEY = '1Nq9AIY3pboYxMTLxMnV1O_YCS5oI1y1LF_dLeiu5fUw' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 361# スロットの1台目
wait = 0.01

user_id = "g84909252+002@gmail.com"
user_pw = "sitega21"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

mobile_emulation = { "deviceName": "iPhone 12 Pro" }
headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
#s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
#s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2000":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2000":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/13', '025579', '1', '32821.0', '6320.0', '18', '4']
['08/13', '025579', '2', '10000.0', '-6018.0', '1', '1']
['08/13', '025579', '3', '15897.0', '2107.0', '9', '2']
['08/13', '025579', '4', '24872.0', '-17754.0', '1', '1']
['08/13', '025436', '5', '11795.0', '1205.0', '12', '3']
['08/13', '025436', '6', '25641.0', '-2106.0', '18', '6']
['08/13', '025489', '7', '16154.0', '-3309.0', '5', '2']
['08/13', '025521', '8', '28462.0', '-5717.0', '22', '12']
['08/13', '025604', '9', '35641.0', '11436.0', '43', '17']
['08/13', '025603', '10', '0', '0', '0', '0']
['08/13', '025605', '11', '18718.0', '6922.0', '26', '9']
['08/13', '025737', '12', '33846.0', '17154.0', '94', '8']
['08/13', '025670', '13', '24359.0', '-8425.0', '21', '3']
['08/13', '025873', '14', '40769.0', '-3309.0', '23', '4']
['08/13', '025680', '15', '37949.0', '-29490.0', '1', '1']
['08/13', '025774', '16', '17692.0', '3913.0', '30', '7']
['08/13', '025839', '17', '23846.0', '9931.0', '25', '3']
['08/13', '025606', '18', 

['08/13', '025180', '144', '4872.0', '603.0', '6', '2']
['08/13', '025665', '145', '27949.0', '-7221.0', '13', '5']
['08/13', '025665', '146', '19744.0', '302.0', '12', '3']
['08/13', '025665', '147', '18718.0', '-13541.0', '3', '3']
['08/13', '025665', '148', '35385.0', '-8124.0', '15', '4']
['08/13', '025665', '149', '32051.0', '-18356.0', '8', '5']
['08/13', '025665', '150', '18974.0', '-9328.0', '5', '2']
['08/13', '025665', '151', '39231.0', '-29791.0', '3', '3']
['08/13', '025665', '152', '24103.0', '31598.0', '32', '5']
['08/13', '025665', '153', '27436.0', '11737.0', '22', '2']
['08/13', '025665', '154', '27692.0', '4515.0', '19', '5']
['08/13', '025687', '155', '26667.0', '-8726.0', '10', '2']
['08/13', '025687', '156', '14872.0', '-10832.0', '1', '1']
['08/13', '025867', '157', '16667.0', '-1203.0', '10', '2']
['08/13', '025867', '158', '18205.0', '904.0', '14', '3']
['08/13', '025867', '159', '15897.0', '-6920.0', '5', '2']
['08/13', '025867', '160', '14103.0', '-10231.0', '

['08/13', '025888', '285', '7949.0', '-6920.0', '0', '0']
['08/13', '025888', '286', '16410.0', '-10832.0', '2', '1']
['08/13', '025888', '287', '26667.0', '-4212.0', '12', '5']
['08/13', '025888', '288', '24872.0', '9631.0', '20', '6']
['08/13', '025579', '289', '16923.0', '3612.0', '11', '1']
['08/13', '025579', '290', '6410.0', '-1203.0', '2', '1']
['08/13', '025579', '291', '14872.0', '-11735.0', '0', '0']
['08/13', '025579', '292', '10513.0', '-6319.0', '1', '1']
['08/13', '025579', '293', '18974.0', '-12939.0', '1', '1']
['08/13', '025579', '294', '13333.0', '-10531.0', '0', '0']
['08/13', '025579', '295', '24359.0', '-1805.0', '10', '2']
['08/13', '025579', '296', '11538.0', '-5717.0', '2', '1']
['08/13', '025579', '297', '27179.0', '7223.0', '16', '4']
['08/13', '025834', '298', '14872.0', '12339.0', '14', '4']
['08/13', '025834', '299', '5641.0', '-4513.0', '0', '0']
['08/13', '025834', '300', '13846.0', '-7522.0', '2', '2']
['08/13', '025834', '301', '5128.0', '-3911.0', '0',

['08/13', '119996', '423', '1052', '-602.0', '2', '1', '--']
['08/13', '120033', '424', '4785', '2500.0', '20', '7', '--']
['08/13', '120033', '425', '4184', '-602.0', '13', '12', '--']
['08/13', '120033', '426', '4326', '1898.0', '21', '6', '--']
['08/13', '119984', '427', '784', '-278.0', '1', '3', '--']
['08/13', '119984', '428', '522', '-324.0', '1', '0', '--']
['08/13', '119984', '429', '677', '-231.0', '2', '2', '--']
['08/13', '120010', '430', '4948', '1435.0', '25', '13', '--']
['08/13', '120010', '431', '1857', '-324.0', '7', '3', '--']
['08/13', '120010', '432', '1491', '-324.0', '5', '3', '--']
['08/13', '120010', '433', '3304', '231.0', '13', '9', '--']
['08/13', '120010', '434', '4545', '-185.0', '17', '11', '--']
['08/13', '120010', '435', '2377', '-509.0', '9', '3', '--']
['08/13', '120010', '436', '2680', '-972.0', '7', '5', '--']
['08/13', '120010', '437', '1964', '324.0', '8', '8', '--']
['08/13', '120010', '438', '5284', '1713.0', '29', '11', '--']
['08/13', '120010'

{'spreadsheetId': '1Nq9AIY3pboYxMTLxMnV1O_YCS5oI1y1LF_dLeiu5fUw',
 'updates': {'spreadsheetId': '1Nq9AIY3pboYxMTLxMnV1O_YCS5oI1y1LF_dLeiu5fUw',
  'updatedRange': 'S20RAW!F31018:M31164',
  'updatedRows': 147,
  'updatedColumns': 8,
  'updatedCells': 1176}}

In [15]:
browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
#s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2000":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2000":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['07/29', '025579', '1', '18974.0', '-1203.0', '8', '3']
['07/29', '025579', '2', '18205.0', '12038.0', '17', '4']
['07/29', '025579', '3', '11538.0', '3913.0', '8', '1']
['07/29', '025579', '4', '25128.0', '-902.0', '13', '2']
['07/29', '025436', '5', '21795.0', '-5717.0', '13', '4']
['07/29', '025436', '6', '13077.0', '7524.0', '18', '4']
['07/29', '025489', '7', '14872.0', '15649.0', '15', '3']
['07/29', '025521', '8', '27179.0', '1205.0', '31', '14']
['07/29', '025604', '9', '28205.0', '3311.0', '31', '16']
['07/29', '025603', '10', '21026.0', '2408.0', '22', '8']
['07/29', '025605', '11', '10000.0', '603.0', '12', '5']
['07/29', '025737', '12', '24615.0', '6621.0', '51', '6']
['07/29', '025670', '13', '28974.0', '-8124.0', '31', '6']
['07/29', '025707', '14', '12308.0', '-1504.0', '7', '2']
['07/29', '025680', '15', '33590.0', '13844.0', '30', '5']
['07/29', '025774', '16', '7692.0', '-3309.0', '5', '4']
['07/29', '025839', '17', '26154.0', '20464.0', '42', '8']
['07/29', '025606'

['07/29', '025718', '143', '9487.0', '-1203.0', '36', '3']
['07/29', '025180', '144', '8205.0', '3612.0', '13', '4']
['07/29', '025665', '145', '36154.0', '-14143.0', '13', '6']
['07/29', '025665', '146', '30000.0', '-1805.0', '16', '3']
['07/29', '025665', '147', '35385.0', '-20462.0', '11', '4']
['07/29', '025665', '148', '22308.0', '9931.0', '20', '4']
['07/29', '025665', '149', '23590.0', '2709.0', '16', '5']
['07/29', '025665', '150', '30000.0', '8427.0', '23', '6']
['07/29', '025665', '151', '32564.0', '-24073.0', '3', '3']
['07/29', '025665', '152', '28718.0', '-16550.0', '5', '1']
['07/29', '025665', '153', '43846.0', '1806.0', '26', '5']
['07/29', '025665', '154', '32821.0', '2709.0', '22', '6']
['07/29', '025687', '155', '11538.0', '-3610.0', '4', '1']
['07/29', '025687', '156', '34359.0', '302.0', '21', '6']
['07/29', '025867', '157', '27179.0', '-4513.0', '12', '4']
['07/29', '025867', '158', '33077.0', '-14143.0', '9', '8']
['07/29', '025867', '159', '37436.0', '3010.0', '

['07/29', '025888', '284', '22564.0', '2408.0', '14', '6']
['07/29', '025888', '285', '17692.0', '-5416.0', '6', '4']
['07/29', '025888', '286', '26154.0', '-4814.0', '11', '4']
['07/29', '025888', '287', '21026.0', '5117.0', '15', '3']
['07/29', '025888', '288', '29487.0', '-23471.0', '1', '1']
['07/29', '025579', '289', '9231.0', '-7221.0', '0', '0']
['07/29', '025579', '290', '12821.0', '603.0', '6', '4']
['07/29', '025579', '291', '18462.0', '-12939.0', '1', '1']
['07/29', '025579', '292', '7436.0', '-3610.0', '2', '1']
['07/29', '025579', '293', '8974.0', '-7221.0', '0', '0']
['07/29', '025579', '294', '14359.0', '28288.0', '24', '2']
['07/29', '025579', '295', '25128.0', '603.0', '12', '4']
['07/29', '025579', '296', '16667.0', '-5717.0', '4', '2']
['07/29', '025579', '297', '24359.0', '-2106.0', '10', '2']
['07/29', '025834', '298', '35385.0', '-20462.0', '2', '1']
['07/29', '025834', '299', '10513.0', '-8124.0', '0', '0']
['07/29', '025834', '300', '5897.0', '-2707.0', '1', '1'

['07/29', '119996', '422', '3668', '1019.0', '18', '11', '--']
['07/29', '119996', '423', '1506', '-139.0', '7', '2', '--']
['07/29', '120033', '424', '4347', '1019.0', '19', '9', '--']
['07/29', '120033', '425', '2181', '-1019.0', '4', '6', '--']
['07/29', '120033', '426', '2704', '1620.0', '13', '6', '--']
['07/29', '119984', '427', '2255', '-463.0', '6', '5', '--']
['07/29', '119984', '428', '658', '-509.0', '0', '3', '--']
['07/29', '119984', '429', '356', '-139.0', '1', '1', '--']
['07/29', '120010', '430', '5985', '-185.0', '20', '22', '--']
['07/29', '120010', '431', '963', '-231.0', '2', '2', '--']
['07/29', '120010', '432', '3891', '139.0', '17', '10', '--']
['07/29', '120010', '433', '1729', '-139.0', '7', '2', '--']
['07/29', '120010', '434', '4461', '-139.0', '17', '14', '--']
['07/29', '120010', '435', '1005', '-463.0', '3', '2', '--']
['07/29', '120010', '436', '6562', '-139.0', '26', '18', '--']
['07/29', '120010', '437', '2137', '-417.0', '6', '4', '--']
['07/29', '1200

{'spreadsheetId': '1Nq9AIY3pboYxMTLxMnV1O_YCS5oI1y1LF_dLeiu5fUw',
 'updates': {'spreadsheetId': '1Nq9AIY3pboYxMTLxMnV1O_YCS5oI1y1LF_dLeiu5fUw',
  'updatedRange': 'S20RAW!F28960:M29106',
  'updatedRows': 147,
  'updatedColumns': 8,
  'updatedCells': 1176}}